<a href="https://colab.research.google.com/github/ucfilho/ANN/blob/master/ANN_2022/Image_04/class_004_image_processing_smart_trash_bin_001.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2
import numpy as np
import pandas as pd
import random
import zipfile
from matplotlib import pyplot as plt
import skimage.feature
from string import digits
import seaborn as sns
import skimage

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
!gdown 1L4mdCDBgybKqr5Wo-kqT8HXcBJ9HXqSX

Downloading...
From: https://drive.google.com/uc?id=1L4mdCDBgybKqr5Wo-kqT8HXcBJ9HXqSX
To: /content/trash_nov_22_2018.zip
100% 42.8M/42.8M [00:00<00:00, 163MB/s]


In [ ]:
FILE='trash_nov_22_2018.zip'

In [ ]:
archive = zipfile.ZipFile('trash_nov_22_2018.zip', 'r')
archive.extractall()

In [ ]:
img_name=[]
with zipfile.ZipFile(FILE, "r") as f:
    for name in f.namelist():
        img_name.append(name)

In [ ]:
Img_Size = 200
ref = 'jpg'
notref = 'met'

In [ ]:
X= []  
Y =[]
n = len(img_name)
i = 0
for i in range(n):
  name = img_name[i]
  if(ref in name):
    Y.append(name)
    X.append(cv2.imread(name,0))
m = len(Y)
print(n,m)

2527 2527


In [ ]:
def properties(glcm_names,img,Img_Size):
  glcm =[]
  resized = cv2.resize(img, (Img_Size,Img_Size))
  m = skimage.exposure.rescale_intensity(resized, in_range='image', out_range='dtype')
  g = skimage.feature.greycomatrix(m, [1, 2], [0, np.pi/2], levels=256,normed=True, symmetric=True)
  for x in glcm_names:
    prop = skimage.feature.greycoprops(g, x)
    glcm.append(np.max(prop))
  
  return glcm

glcm_names =['contrast', 'dissimilarity', 'homogeneity', 'energy', 'correlation', 'ASM']

# see https://github.com/ucfilho/ANN/blob/master/ANN_2022/Image/class_001_image_processing_smart_trash_bin_012.ipynb
features =[]
for i in range(len(img_name)):
  glcm = []
  img = X[i]
  glcm = properties(glcm_names,img,Img_Size)
  
  img = cv2.imread(img_name[i])
  green_img = img[:,:,1]
  values_green = properties(glcm_names,green_img,Img_Size)

  red_img = img[:,:,0]
  values_red = properties(glcm_names,red_img,Img_Size)

  blue_img = img[:,:,2]
  values_blue = properties(glcm_names,blue_img,Img_Size)

  total= [*glcm,*values_red, *values_green, *values_blue]
  
  features.append(total)



In [ ]:
lista = []
glcm_names =['contrast', 'dissimilarity', 'homogeneity', 'energy', 'correlation', 'ASM']
names = glcm_names
for x in glcm_names:
  lista.append(x+'_r')
names = glcm_names + lista

lista = []
for x in glcm_names:
  lista.append(x+'_g')
names = names + lista

lista = []
for x in glcm_names:
  lista.append(x+'_b')
names = names + lista

print(names)

df = pd.DataFrame(features,columns=names)

['contrast', 'dissimilarity', 'homogeneity', 'energy', 'correlation', 'ASM', 'contrast_r', 'dissimilarity_r', 'homogeneity_r', 'energy_r', 'correlation_r', 'ASM_r', 'contrast_g', 'dissimilarity_g', 'homogeneity_g', 'energy_g', 'correlation_g', 'ASM_g', 'contrast_b', 'dissimilarity_b', 'homogeneity_b', 'energy_b', 'correlation_b', 'ASM_b']


In [ ]:
glcm_names

['contrast', 'dissimilarity', 'homogeneity', 'energy', 'correlation', 'ASM']

In [ ]:
n =len(img_name)
tipos = []
selected = 'find'
for x in img_name:
  result =''.join([i for i in x[:-4] if not i.isdigit()])
  if(result != selected):
    selected = result
    tipos.append(selected)


In [ ]:
label =[]
for name in img_name:
  for idx, x in enumerate(tipos):
    if(x in name):
      label.append(idx)

In [ ]:
df['target']=label

In [ ]:
for i in range(6): 
  df_plot= df[(df["target"] == i)]
  n= df_plot.shape[0]
  print(tipos[i],n)

cardboard 403
glass 501
metal 410
paper 594
plastic 482
trash 137


In [ ]:
y=df['target']
X=df.drop(['target'], axis='columns')

In [ ]:
y = pd.get_dummies(y)
# check https://pandas.pydata.org/docs/reference/api/pandas.get_dummies.html
y

,0,1,2,3,4,5
0,1,0,0,0,0,0
1,1,0,0,0,0,0
2,1,0,0,0,0,0
3,1,0,0,0,0,0
4,1,0,0,0,0,0
...,...,...,...,...,...,...
2522,0,0,0,0,0,1
2523,0,0,0,0,0,1
2524,0,0,0,0,0,1
2525,0,0,0,0,0,1


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, stratify=y, random_state=1)

In [ ]:
scaler = StandardScaler()
# transform data
scaler_X = scaler.fit(X_train)
X_train = scaler_X.transform(X_train)
X_test = scaler_X.transform(X_test)

In [ ]:
nin = X_train.shape[1]
nout =y.shape[1]
nhidden = 30

In [ ]:
import tensorflow as tf

In [ ]:
# Create a `Sequential` model and add a Dense layer as the first layer.
model = tf.keras.models.Sequential()
model.add(tf.keras.Input(shape=(nin,)))
model.add(tf.keras.layers.Dense(nhidden, activation='relu'))
# Now the model will take as input arrays of shape (None, 16)
# and output arrays of shape (None, 32).
# Note that after the first layer, you don't need to specify
# the size of the input anymore:
model.add(tf.keras.layers.Dense(nout, activation='softmax'))


In [ ]:
X_test = np.array(X_test )
y_test = np.array(y_test)
y_test = y_test.astype(np.int32)
y_train = y_train.astype(np.int32)
X_train = X_train.reshape(-1, nin) 
X_test = X_test.reshape(-1, nin)

In [ ]:
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [ ]:
# gives us back a <keras.callbacks.History object at 0x112e61a90>
#r = model.fit(X_train, y_train, validation_data = (X_test,y_test), epochs=15, batch_size=32)
r = model.fit(X_train, y_train, epochs=1000)
print("Returned:", r)

Epoch 1/1000
53/53 [==============================] - 1s 2ms/step - loss: 1.8090 - accuracy: 0.2079
Epoch 2/1000
53/53 [==============================] - 0s 2ms/step - loss: 1.5095 - accuracy: 0.4082
Epoch 3/1000
53/53 [==============================] - 0s 2ms/step - loss: 1.4313 - accuracy: 0.4400
Epoch 4/1000
53/53 [==============================] - 0s 2ms/step - loss: 1.3911 - accuracy: 0.4430
Epoch 5/1000
53/53 [==============================] - 0s 2ms/step - loss: 1.3598 - accuracy: 0.4536
Epoch 6/1000
53/53 [==============================] - 0s 2ms/step - loss: 1.3374 - accuracy: 0.4560
Epoch 7/1000
53/53 [==============================] - 0s 2ms/step - loss: 1.3164 - accuracy: 0.4773
Epoch 8/1000
53/53 [==============================] - 0s 2ms/step - loss: 1.2982 - accuracy: 0.4808
Epoch 9/1000
53/53 [==============================] - 0s 2ms/step - loss: 1.2812 - accuracy: 0.4867
Epoch 10/1000
53/53 [==============================] - 0s 2ms/step - loss: 1.2675 - accuracy: 0.4861

In [ ]:
# usa a rede neural treina em calculos para treino e teste
y_calc_train=model.predict(X_train)
y_calc_test=model.predict(X_test)

In [ ]:
y_calc_test

array([[8.33699405e-02, 3.56450822e-04, 1.80682255e-05, 9.14179087e-01,
        1.96709158e-03, 1.09422501e-04],
       [1.18517568e-02, 4.81176347e-01, 5.40815033e-02, 2.69331127e-01,
        1.07266679e-01, 7.62925670e-02],
       [3.21592117e-04, 1.71158612e-01, 1.80675574e-02, 9.90470033e-03,
        7.68052518e-01, 3.24950442e-02],
       ...,
       [5.03829382e-02, 5.27966619e-01, 1.95278928e-01, 1.33550679e-02,
        1.05737425e-01, 1.07279085e-01],
       [4.25126258e-04, 1.49281254e-06, 8.41169851e-04, 9.97926593e-01,
        5.36282059e-05, 7.52001826e-04],
       [1.84523780e-02, 4.92538065e-01, 2.55770981e-03, 3.02915782e-01,
        1.31174639e-01, 5.23613803e-02]], dtype=float32)

In [ ]:
def retorna(y):
  resp = []
  for yi in y:
    vmax =max(yi)
    for idx, x in enumerate(yi):
      if(x == vmax):
        resp.append(idx)
  return resp

In [ ]:
y_test[0]

In [ ]:
pd.__version__

In [ ]:

y_calc = retorna(y_calc_test)
y_test = retorna(y_test)

In [ ]:
confusion_matrix(y_test, y_calc)

array([[105,   5,   6,  13,   4,   0],
       [  1,  92,  31,  13,  28,   1],
       [  2,  27,  74,  18,  12,   2],
       [  8,  13,  12, 138,  25,   0],
       [  6,  27,   7,  27,  92,   0],
       [  3,  10,   8,   9,   6,   9]])

In [ ]:
# Classification Report
print('\n\n', classification_report(y_test, y_calc, target_names=tipos))



               precision    recall  f1-score   support

   cardboard       0.84      0.79      0.81       133
       glass       0.53      0.55      0.54       166
       metal       0.54      0.55      0.54       135
       paper       0.63      0.70      0.67       196
     plastic       0.55      0.58      0.56       159
       trash       0.75      0.20      0.32        45

    accuracy                           0.61       834
   macro avg       0.64      0.56      0.57       834
weighted avg       0.62      0.61      0.61       834



In [ ]:
''''
Img_Size = 80 hidden = 12

               precision    recall  f1-score   support

   cardboard       0.78      0.74      0.76       133
       glass       0.51      0.41      0.45       166
       metal       0.45      0.46      0.45       135
       paper       0.58      0.68      0.63       196
     plastic       0.49      0.59      0.54       159
       trash       0.47      0.18      0.26        45

    accuracy                           0.56       834
   macro avg       0.55      0.51      0.51       834
weighted avg       0.55      0.56      0.55       834

'''

In [ ]:
'''
Img_Size = 100

               precision    recall  f1-score   support

   cardboard       0.79      0.74      0.76       133
       glass       0.45      0.40      0.42       166
       metal       0.45      0.44      0.44       135
       paper       0.59      0.66      0.62       196
     plastic       0.48      0.58      0.53       159
       trash       0.59      0.22      0.32        45

    accuracy                           0.55       834
   macro avg       0.56      0.51      0.52       834
weighted avg       0.55      0.55      0.54       834
'''

In [ ]:
'''
Img_Size = 200
               precision    recall  f1-score   support

   cardboard       0.82      0.76      0.79       133
       glass       0.46      0.46      0.46       166
       metal       0.44      0.41      0.42       135
       paper       0.61      0.69      0.65       196
     plastic       0.51      0.55      0.53       159
       trash       0.36      0.18      0.24        45

    accuracy                           0.56       834
   macro avg       0.53      0.51      0.51       834
weighted avg       0.55      0.56      0.55       834
'''

In [ ]:
# Classification Report
print('\n\n', classification_report(y_test, y_test, target_names=tipos))